# Importación de librerías
=============================================================================================================

In [1]:
import pandas as pd
import numpy as np
import os

import json
import glob
import gzip
import pickle

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

import pyarrow.parquet as pq
import pyarrow as pa
from google.cloud import storage

import math
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
from math import factorial
from scipy import stats as st


In [2]:
# Se eliminan advertencias
import warnings

# Suprimir las advertencias SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [3]:
# Se crea una función para subir los dataframes preprocesados a Google Cloud Storage

# Configura tus credenciales de Google Cloud
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/credentials.json'

# Función que sube el archivo a Google Cloud Storage
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Sube un archivo a un bucket de Google Cloud Storage."""
    # Inicializa el cliente de GCS
    storage_client = storage.Client()
    # Obtén el bucket
    bucket = storage_client.bucket(bucket_name)
    # Crea un blob en el bucket
    blob = bucket.blob(destination_blob_name)
    # Sube el archivo al blob
    blob.upload_from_filename(source_file_name)
    print(f"Archivo {source_file_name} subido a {destination_blob_name}.")

# Función que guarda el dataframe en un archivo temporal de parquet y lo sube
def dataframe_to_parquet_and_upload(df, bucket_name, destination_blob_name):
    # Guarda el DataFrame como archivo Parquet
    table = pa.Table.from_pandas(df)
    pq.write_table(table, 'temp.parquet')

    # Sube el archivo Parquet a Google Cloud Storage
    upload_to_gcs(bucket_name, 'temp.parquet', destination_blob_name)

    # Elimina el archivo temporal
    os.remove('temp.parquet')

In [4]:
# Nombre del bucket de GCS
bucket_name = 'archivos-preprocesados-henry'

# ETL de Google - 1

## Carga de datos
=============================================================================================================

In [3]:
# Ruta a la ubicación de los archivos JSON
ruta_directorio = '../Data/Google/metadata-sitios'

dataframes = []
for filename in os.listdir(ruta_directorio):        
    if filename.endswith('.json'):
        df_temporal = pd.read_json(os.path.join(ruta_directorio, filename), lines=True)
        dataframes.append(df_temporal)

dfg_sites = pd.concat(dataframes, ignore_index=True)        

In [4]:
print(dfg_sites.info())
dfg_sites.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025011 entries, 0 to 3025010
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   name              object 
 1   address           object 
 2   gmap_id           object 
 3   description       object 
 4   latitude          float64
 5   longitude         float64
 6   category          object 
 7   avg_rating        float64
 8   num_of_reviews    int64  
 9   price             object 
 10  hours             object 
 11  MISC              object 
 12  state             object 
 13  relative_results  object 
 14  url               object 
dtypes: float64(3), int64(1), object(11)
memory usage: 346.2+ MB
None


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1292896,Road Star Auto Sales,"Road Star Auto Sales, 14910 Meridian Ave E, Pu...",0x5490fda53aad9069:0x14e3e4fc44c0672c,None,47.120693,-122.293661,[Car dealer],4.7,45,None,"[[Monday, 10AM–6PM], [Tuesday, 10AM–6PM], [Wed...",None,Closed ⋅ Opens 10AM Tue,"[0x5490fd3c79d2ea5b:0x8bf353cb4df64a65, 0x5490...",https://www.google.com/maps/place//data=!4m2!3...
738208,Dry Clean Best,"Dry Clean Best, 605 E El Camino Real # 2, Sunn...",0x808fb670d017dcdd:0x7e179ae4e11a40ef,None,37.361912,-122.024475,[Dry cleaner],4.4,8,None,"[[Tuesday, 7AM–6PM], [Wednesday, 7AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7AM,"[0x808fb661537793d7:0xa24aeb299ba4ac98, 0x808f...",https://www.google.com/maps/place//data=!4m2!3...


## Exportación de datos
=========================================================

In [5]:
# dfg_sites.to_csv('dfg_sites.csv', index=False)
dfg_sites.to_parquet('../Data/Google/reviews-estados-parquet/dfg_sites.parquet')

## Obtención de datos de los restaurantes
=========================================================

In [6]:
# Carga del dataframe de Sites
dfg_sites = pd.read_parquet('../Data/Google/reviews-estados-parquet/dfg_sites.parquet')

In [7]:
# Se obtiene la categoría de los negocios.
negocios_cat = dfg_sites[["gmap_id","category"]].dropna()
# Se separan las categorías de los negocios
negocios_cat = negocios_cat.explode("category")
negocios_cat.head()

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,Textile exporter
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
3,0x80c2c89923b27a41:0x32041559418d447,Fabric store
4,0x80c2c632f933b073:0xc31785961fe826a6,Fabric store


In [8]:
# Se observa la distribución de las categorías.
negocios_cat["category"].value_counts(normalize=True).head(15)

category
Restaurant               0.015530
Auto repair shop         0.012431
Gas station              0.011200
Service establishment    0.010615
Beauty salon             0.010493
Convenience store        0.009926
Church                   0.008474
Hair salon               0.008266
Nail salon               0.007121
ATM                      0.006779
Park                     0.006772
Doctor                   0.005855
Clothing store           0.005823
Dentist                  0.005770
Insurance agency         0.005697
Name: proportion, dtype: float64

In [9]:
# Se obtienen los id de los negocios cuyas categorías incluyan restaurantes o bares.
id_rest = negocios_cat[negocios_cat["category"].str.contains('restaurant|bar', case=False)]
id_rest.head(10)

,gmap_id,category
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
8,0x87ec235c54d25b31:0x3b75fb5facc602f,Health food restaurant
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Restaurant
75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,Bar
87,0x88e711bd2244fe3b:0x8406dd780f0574d1,Barber shop
96,0x7c00456eecad3111:0x8217f9600c51f33,Restaurant
111,0x89a911d03136ef51:0xb1c06825101afe51,Barber shop
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Restaurant
115,0x88c2e4e34f1ed783:0x76c5da381c499d79,Buffet restaurant


In [10]:
# Se observan los elementos que contienen "bar" dentro de su categoría para identificar aquellos que no corresponden o pertenecen a la categoría de negocios que se espera.
id_rest[id_rest["category"].str.contains("bar",case=False)]["category"].unique()

array(['Bar', 'Barber shop', 'Cocktail bar', 'Bar & grill', 'Sports bar',
       'Wine bar', 'Bark supplier', 'Barbecue restaurant', 'Snack bar',
       'Barber supply store', 'Gay bar', 'Espresso bar', 'Eyebrow bar',
       'Hookah bar', 'Karaoke bar', 'Barn', 'Bartending school',
       'Oyster bar restaurant', 'Poke bar', 'Tapas bar', 'Barber school',
       'Piano bar', 'Subaru dealer', 'Barrister', 'Bar stool supplier',
       'Dart bar', 'Live music bar', 'Korean barbecue restaurant',
       'Cider bar', 'Mongolian barbecue restaurant', 'Cabaret club',
       'Barbecue area', 'Hyperbaric medicine physician',
       'Bar restaurant furniture store', 'Barrel supplier',
       'Bariatric surgeon', 'Bar PMU', 'Salsa bar', 'Military barracks',
       'Tiki bar', 'Stand bar', 'Bar tabac', 'Army barracks',
       'Offal barbecue restaurant', 'Cured ham bar', 'Girl bar',
       'Mutton barbecue restaurant'], dtype=object)

In [11]:
# Se crea una lista con los elementos que no pertenecen a la categoría de negocios esperada.
no = ['Barber shop', 'Bark supplier',
       'Barber supply store', 'Eyebrow bar',
       'Hookah bar', 'Bartending school',
       'Barber school', 'Subaru dealer', 'Barrister', 'Bar stool supplier',
       'Cabaret club', 'Barbecue area', 'Hyperbaric medicine physician',
       'Bar restaurant furniture store', 'Barrel supplier',
       'Bariatric surgeon', 'Military barracks', 'Army barracks']

In [12]:
# Se filtra nuevamente el dataset de las categorías para obtener únicamente las categorías de los negocios deseadas
id_rest_final = id_rest[~id_rest["category"].isin(no)]
id_rest_final.head(5)

,gmap_id,category
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
8,0x87ec235c54d25b31:0x3b75fb5facc602f,Health food restaurant
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Restaurant
75,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,Bar


In [13]:
# Se observa que las categorías de negocios se encuentren dentro de lo esperado (bares y restaurantes).
id_rest_final["category"].unique()

array(['Korean restaurant', 'Restaurant', 'Health food restaurant', 'Bar',
       'Buffet restaurant', 'Cocktail bar', 'Fast food restaurant',
       'Family restaurant', 'Seafood restaurant', 'Dominican restaurant',
       'Taco restaurant', 'Mexican restaurant', 'Restaurant or cafe',
       'Bar & grill', 'Breakfast restaurant', 'Lunch restaurant',
       'Asian restaurant', 'Hot pot restaurant', 'Chinese restaurant',
       'Mediterranean restaurant', 'Takeout Restaurant', 'Sports bar',
       'Italian restaurant', 'Wine bar', 'Ramen restaurant',
       'American restaurant', 'Modern French restaurant',
       'Pizza restaurant', 'Vegetarian restaurant', 'Delivery Restaurant',
       'Sushi restaurant', 'Japanese restaurant', 'Creole restaurant',
       'Caribbean restaurant', 'Barbecue restaurant', 'Snack bar',
       'Hamburger restaurant', 'Soul food restaurant',
       'Burrito restaurant', 'Gay bar', 'Latin American restaurant',
       'Spanish restaurant', 'Cheesesteak restaur

In [15]:
# Se filtra el data de los negocios para obtener aquellos que son de negocios de restaurantes o bares.
dfg_rest = dfg_sites[dfg_sites["gmap_id"].isin(id_rest_final["gmap_id"])]
dfg_rest.head(10)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,None,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 9:30PM,None,https://www.google.com/maps/place//data=!4m2!3...
8,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,None,41.616079,-93.865487,"[Bakery, Health food restaurant]",4.7,21,None,"[[Thursday, 10AM–5:30PM], [Friday, 10AM–5:30PM...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,"[0x87ee974869295555:0x95f310d065882c9b, 0x87ec...",https://www.google.com/maps/place//data=!4m2!3...
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...
75,The Bar Stool,"The Bar Stool, 618 Main St, Highland, WI 53543",0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,None,43.046428,-90.380139,[Bar],4.5,18,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,"[0x87fd0e70dc5eeb3d:0x7f3ce1daba0a5f24, 0x87fd...",https://www.google.com/maps/place//data=!4m2!3...
96,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,[Restaurant],4.4,18,None,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...
114,Cape Seafood Shack,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,None,26.641377,-81.940545,[Restaurant],5.0,1,$$,None,{'Accessibility': ['Wheelchair accessible entr...,None,None,https://www.google.com/maps/place//data=!4m2!3...
115,Fresh Point Country Buffet,"Fresh Point Country Buffet, 10525 US-19, Pinel...",0x88c2e4e34f1ed783:0x76c5da381c499d79,None,27.867489,-82.702971,[Buffet restaurant],5.0,2,None,"[[Thursday, 11AM–9PM], [Friday, 11AM–9PM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Permanently closed,"[0x88c2e5593fbd022b:0x2ccd7a719849c9f7, 0x88c2...",https://www.google.com/maps/place//data=!4m2!3...
123,Golden Castle,"Golden Castle, 1906 E 12th St, Austin, TX 78702",0x8644b59b8fe872e5:0x5e638876caa84cc3,None,30.273985,-97.719563,[Restaurant],4.5,8,None,"[[Thursday, 5PM–12AM], [Friday, 5PM–12AM], [Sa...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 5PM,None,https://www.google.com/maps/place//data=!4m2!3...
126,Onyx Bar,"Onyx Bar, 11011 W Charleston Blvd, Las Vegas, ...",0x80c8be4e73e8263f:0x3edb275a351e6266,None,36.156205,-115.332636,[Cocktail bar],4.7,8,None,"[[Thursday, 4–11PM], [Friday, 4PM–1AM], [Satur...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 4PM,"[0x80c8be531e3b3471:0xb681abb0b8d07596, 0x80c8...",https://www.google.com/maps/place//data=!4m2!3...


In [16]:
# Se guarda la base de restaurantes y bares.
dfg_rest.to_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

## Conclusiones 
========================================================

* Se realiza la lectura, concatenación y exportación de los datos de locales de google
* Debido a la cuantiosa cantidad de información, el preprocesamiento de estos datos se realiza en el EDA

# ETL de Google - 2

## Carga de datos 
==================================================================

In [17]:
# Sites
dfg_rest = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

# Reviews
# Este dataset se carga más adelante en su sección de pre-procesamiento

## Preprocesamiento
============================================================

### Dataset SITES

In [18]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              248852 non-null  object 
 1   address           247751 non-null  object 
 2   gmap_id           248852 non-null  object 
 3   description       79018 non-null   object 
 4   latitude          248852 non-null  float64
 5   longitude         248852 non-null  float64
 6   category          248852 non-null  object 
 7   avg_rating        248852 non-null  float64
 8   num_of_reviews    248852 non-null  int64  
 9   price             115831 non-null  object 
 10  hours             220683 non-null  object 
 11  MISC              246706 non-null  object 
 12  state             221749 non-null  object 
 13  relative_results  203743 non-null  object 
 14  url               248852 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 30.4+ MB
None


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1496641,CALVIN'S KOREAN HOT CHICKEN,"CALVIN'S KOREAN HOT CHICKEN, 3211 Holiday Ct S...",0x80dc07e35b78cb4f:0x96beb1a46c73711e,"Compact, informal destination with outdoor sea...",32.870019,-117.233508,"[Fried chicken takeaway, Chicken shop, Chicken...",5.0,46,None,"[[Tuesday, 11AM–8PM], [Wednesday, 11AM–8PM], [...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 11AM Wed,None,https://www.google.com/maps/place//data=!4m2!3...
2668084,Berwick Family Resturaunt,"Berwick Family Resturaunt, 110 N Mulberry St, ...",0x89c59d0325812891:0x6fc3f2d46857fbaf,None,41.054444,-76.235556,[Family restaurant],4.3,138,None,"[[Sunday, 6AM–2PM], [Monday, 8AM–2PM], [Tuesda...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 2PM,"[0x89c59d1a4ccc1899:0xe605a9205d997bbb, 0x89c5...",https://www.google.com/maps/place//data=!4m2!3...


In [19]:
# Adecuación de tipos de dato
dfg_rest = dfg_rest[['name', 'address', 'gmap_id', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price']]
dfg_rest = dfg_rest.rename(columns={'address': 'address_full'})
dfg_rest[['address', 'city', 'postal_code']] = dfg_rest['address_full'].str.extract(r'.*,\s*([^,]+),\s*([^,]+),\s*([^,]+)')
dfg_rest['state'] = dfg_rest['postal_code'].str.split(' ').str[0]
dfg_rest['postal_code'] = dfg_rest['postal_code'].str.split(' ').str[1]

dfg_rest['postal_code'] = pd.to_numeric(dfg_rest['postal_code'], errors='coerce')
dfg_rest = dfg_rest.dropna(subset=['postal_code'])
dfg_rest['postal_code'] = dfg_rest['postal_code'].astype(int)

In [20]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 231427 entries, 2 to 3025006
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            231427 non-null  object 
 1   address_full    231427 non-null  object 
 2   gmap_id         231427 non-null  object 
 3   latitude        231427 non-null  float64
 4   longitude       231427 non-null  float64
 5   avg_rating      231427 non-null  float64
 6   num_of_reviews  231427 non-null  int64  
 7   price           104084 non-null  object 
 8   address         231427 non-null  object 
 9   city            231427 non-null  object 
 10  postal_code     231427 non-null  int32  
 11  state           231427 non-null  object 
dtypes: float64(3), int32(1), int64(1), object(7)
memory usage: 22.1+ MB
None


,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2101489,Difrego's Hide Out Charcoal Grill Kitchen & Pizza,Difrego's Hide Out Charcoal Grill Kitchen & Pi...,0x89c3bf603982c725:0x70b8bc0dd32dd5b4,40.554275,-74.524840,3.7,58,None,77 S Main St,South Bound Brook,8880,NJ
1109016,Zab Thai Food Kitchen,"Zab Thai Food Kitchen, 16720 E Ave of the Foun...",0x872b99f02649ba2f:0x1f7cf9580074a2c9,33.604968,-111.719693,4.4,25,None,16720 E Ave of the Fountains,Fountain Hills,85268,AZ


In [21]:
# Convertir precio a formato numerico
price_mapping = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}
dfg_rest['price'] = dfg_rest['price'].map(price_mapping)

# Reemplazar valores nulos del campo "precio" con el promedio de precios basado en cercanía, utilzando el código postal
dfg_rest['postal_code'] = dfg_rest['postal_code'].astype(str)
average_price_by_zip = dfg_rest.groupby('postal_code')['price'].apply(lambda x: x.dropna().astype(float).mean())
dfg_rest['price'] = dfg_rest.apply(lambda row: average_price_by_zip.get(row['postal_code']) if pd.isnull(row['price']) else row['price'], axis=1)

dfg_rest = dfg_rest.dropna(subset=['price'])

In [22]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            223485 non-null  object 
 1   address_full    223485 non-null  object 
 2   gmap_id         223485 non-null  object 
 3   latitude        223485 non-null  float64
 4   longitude       223485 non-null  float64
 5   avg_rating      223485 non-null  float64
 6   num_of_reviews  223485 non-null  int64  
 7   price           223485 non-null  float64
 8   address         223485 non-null  object 
 9   city            223485 non-null  object 
 10  postal_code     223485 non-null  object 
 11  state           223485 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 22.2+ MB
None


,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2954365,Pizza Hut,"Pizza Hut, 2601 Castle Hayne Rd, Wilmington, N...",0x89a98a995db410b7:0x27606ef77269faa3,34.289313,-77.923051,3.9,148,1.000000,2601 Castle Hayne Rd,Wilmington,28401,NC
40757,Banh Mi - Juice and Sandwich,"Banh Mi - Juice and Sandwich, 2731 FM 1463 #80...",0x86412380b6f37c41:0x469da66a0a137bc1,29.750890,-95.850806,4.3,28,1.382353,2731 FM 1463 #800,Katy,77494,TX


In [26]:
# Subida del dataset a GCS
destino = 'dfg_rest.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

#dfg_rest.to_parquet('dfg_rest.parquet')

Archivo temp.parquet subido a dfg_rest.parquet.


#### Creacion del dataset de categorias

In [29]:
dfg_rest.head()

,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,4.4,18,1.411765,761 S Vermont Ave,Los Angeles,90005,CA
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,4.5,18,1.333333,12740 La Mirada Blvd,La Mirada,90638,CA
8,"Sweet Rewards Gluten Free Bakery, LLC","Sweet Rewards Gluten Free Bakery, LLC, 85 NE D...",0x87ec235c54d25b31:0x3b75fb5facc602f,41.616079,-93.865487,4.7,21,1.500000,85 NE Dartmoor Dr,Waukee,50263,IA
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,4.0,8,1.581818,701 S 5th St,Philadelphia,19147,PA
75,The Bar Stool,"The Bar Stool, 618 Main St, Highland, WI 53543",0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,43.046428,-90.380139,4.5,18,1.000000,618 Main St,Highland,53543,WI


In [30]:
dfg_restaurants = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")
dfg_categories = dfg_restaurants[['gmap_id', 'category']]
dfg_categories = dfg_categories.explode('category')
dfg_categories.rename(columns={'gmap_id': 'site_id'}, inplace=True)

dfg_categories.info()
dfg_categories.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 626995 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   site_id   626995 non-null  object
 1   category  626995 non-null  object
dtypes: object(2)
memory usage: 14.4+ MB


,site_id,category
441362,0x89c2fdad033831ab:0x7b53ace9a1db0b8e,Cocktail bar
636921,0x86456c86118a668f:0xdc3ac1088a2d1d71,Bar
1609424,0x89c3b10b567f818d:0x6eec95cec36523ce,Restaurant
767957,0x866f94219f0ecf8d:0xac7168f5b68d3b26,Bar
2301257,0x808fc7556dac75b7:0xcecc805f4ac5f192,Pizza delivery
2937141,0x88f5af4b9e73cbfd:0x60ffec905c4a188f,Cookie shop
585482,0x865c667da63be485:0x65a6f731c4a8d363,Bar
2102017,0x89c808f3141a34ed:0x23d8d02f51f3a878,Breakfast restaurant
2416913,0x88351bc9dc742ab1:0xc898b37481f7d862,Mexican restaurant
1779069,0x8091e232b91fc16d:0xb0fa83c9a8c4eddd,Frozen yogurt shop


In [31]:
dfg_categories_grouped = dfg_categories.groupby(['site_id'])['category'].count().reset_index()
dfg_categories_grouped.sort_values(by='category', ascending=False, inplace=True)
dfg_categories_grouped.head()

,site_id,category
24829,0x808f7792bd0c683d:0xefefd9d96f8fcb11,22
100420,0x87c5696c2304eb11:0xf60dbc11df643d20,22
211649,0x89c28801576a92a1:0xe8183528656d5f08,22
224911,0x89c6cf1e78429c4d:0xffccfa7176afdc7e,20
224636,0x89c6c9d5df403af7:0xa304c9f7a08ac8e3,20


In [32]:
# Subida del dataset a GCS
destino = 'dfg_categories.parquet'
dataframe_to_parquet_and_upload(dfg_categories,bucket_name,destino)

# Exportar
#dfg_categories.to_parquet('dfg_site_categories.parquet')
#dfg_categories.to_excel('dfg_site_categories.xlsx')

Archivo temp.parquet subido a dfg_categories.parquet.


#### Creacion del dataset de atributos

In [102]:
dfg_restaurants = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")
dfg_misc = dfg_restaurants[['gmap_id', 'MISC']]

dfg_misc.rename(columns={'gmap_id': 'site_id'}, inplace=True)

dfg_misc.info()
dfg_misc.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   site_id  248852 non-null  object
 1   MISC     246706 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB


,site_id,MISC
2936153,0x808597401e9cce0b:0x2fa3913a4ee0a0f5,{'Accessibility': ['Wheelchair accessible entr...
786761,0x5490405e1bc2154f:0x47135c4f9c180cb0,{'Accessibility': ['Wheelchair accessible entr...
626309,0x89c25b9ee7a61549:0x1d06437c09f03255,"{'Accessibility': None, 'Activities': None, 'A..."
496792,0x89c770e6a46b2269:0xe7f784a5bfa9cef9,{'Accessibility': ['Wheelchair accessible entr...
2529289,0x876c834b3898c7a1:0x3928934627074542,{'Accessibility': ['Wheelchair accessible entr...
559883,0x87fe0f46f4afc4e9:0xa19bdc26f6c6ada,{'Accessibility': ['Wheelchair accessible entr...
2620053,0x8640c24afa9c9113:0x8d2aa3471e393f0f,{'Accessibility': ['Wheelchair-accessible car ...
2799049,0x88fef2d35190652f:0xd27156a488265c75,{'Accessibility': ['Wheelchair accessible entr...
621909,0x880653d9379f5ed3:0x8ffc3fe359f96c34,"{'Accessibility': None, 'Activities': None, 'A..."
821652,0x863547a1d42e5575:0x9b4d57b38487399a,"{'Accessibility': None, 'Activities': None, 'A..."


In [104]:
# Se obtiene un dataframe con los datos normalizados de "MISC"
x = pd.json_normalize(dfg_misc["MISC"])

#Se concatenan los dos dataframes.
dfg_misc = pd.concat([dfg_misc.reset_index(drop=True), x.reset_index(drop=True)], axis=1)

# Se elimina la columna "MISC"
dfg_misc.drop(columns=["MISC"],inplace=True)

# Se visualiza el resultado.
dfg_misc.head(5)

,site_id,Accessibility,Activities,Amenities,Atmosphere,Crowd,Dining options,From the business,Getting here,Health & safety,Health and safety,Highlights,Lodging options,Offerings,Payments,Planning,Popular for,Recycling,Service options
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,[Wheelchair accessible entrance],None,[Good for kids],[Casual],None,None,None,NaN,None,None,None,NaN,[Comfort food],None,None,None,None,"[Takeout, Dine-in, Delivery]"
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,[Wheelchair accessible entrance],None,[Good for kids],[Casual],[College students],"[Lunch, Dinner]",None,NaN,"[Mask required, Staff wear masks, Staff requir...",None,None,NaN,"[Comfort food, Quick bite, Small plates]",None,None,"[Lunch, Dinner, Solo dining]",None,"[Outdoor seating, Curbside pickup, No-contact ..."
2,0x87ec235c54d25b31:0x3b75fb5facc602f,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,[Delivery]
3,0x89c6c89efcaed69d:0xded973f6033e7dba,None,None,[Good for kids],[Casual],"[College students, Locals]",None,None,NaN,None,None,None,NaN,None,None,None,None,None,[Delivery]
4,0x87fd0e70c5f5d87b:0xdf340eeb75040ef3,None,None,[Restroom],"[Casual, Cozy]",[Groups],None,None,NaN,None,None,"[Fast service, Great beer selection, Sports]",NaN,"[Alcohol, Beer, Hard liquor]",None,None,None,None,"[Dine-in, Delivery]"


In [120]:
# Se procede a pasar los datos de las columnas accesibility, activities y amenities a diferentes dataframes 
dfg_attributes_accessibility = dfg_misc[["site_id","Accessibility"]]
dfg_attributes_activities = dfg_misc[["site_id","Activities"]]
dfg_attributes_amenities = dfg_misc[["site_id","Amenities"]]

# Se eliminan datos nulos
dfg_attributes_accessibility.dropna(inplace=True)
dfg_attributes_activities.dropna(inplace=True)
dfg_attributes_amenities.dropna(inplace=True)

# Se explotan los datos dentro de las columnas de las categorías
dfg_attributes_accessibility = dfg_attributes_accessibility.explode("Accessibility")
dfg_attributes_activities = dfg_attributes_activities.explode("Activities")
dfg_attributes_amenities = dfg_attributes_amenities.explode("Amenities")

# Se resetean índices para asegurarse de que sean únicos
dfg_attributes_accessibility = dfg_attributes_accessibility.reset_index(drop=True)
dfg_attributes_activities = dfg_attributes_activities.reset_index(drop=True)
dfg_attributes_amenities = dfg_attributes_amenities.reset_index(drop=True)

In [123]:
# Se resetean índices para asegurarse de que sean únicos
dfg_attributes_accessibility = dfg_attributes_accessibility.rename(columns={"Accessibility":"attributes"})
dfg_attributes_activities = dfg_attributes_activities.rename(columns={"Activities":"attributes"})
dfg_attributes_amenities = dfg_attributes_amenities.rename(columns={"Amenities":"attributes"})

In [130]:
# Se unen los datos en el mismo dataframe
dfg_attributes = pd.concat([dfg_attributes_accessibility, 
                         dfg_attributes_activities, 
                         dfg_attributes_amenities], axis=0, ignore_index=True)
dfg_attributes.head()

,site_id,attributes
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Wheelchair accessible entrance
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Wheelchair accessible entrance
2,0x7c00456eecad3111:0x8217f9600c51f33,Wheelchair accessible entrance
3,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Wheelchair accessible entrance
4,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Wheelchair accessible parking lot


In [132]:
# Subida del dataset a GCS
destino = 'dfg_attributes.parquet'
dataframe_to_parquet_and_upload(dfg_categories,bucket_name,destino)

# dfg_attributes.head()
# dfg_attributes.to_csv("dfg_attributes.csv")

Archivo temp.parquet subido a dfg_attributes.parquet.


#### Creacion del dataset de precios

In [136]:
dfg_rest_prices_by_zip = dfg_rest.groupby('postal_code')['price'].apply(lambda x: x.dropna().astype(float).mean())

dfg_rest_prices_by_zip.reset_index()

,postal_code,price
0,1,1.000000
1,10001,1.639344
2,10002,1.710744
3,10003,1.828025
4,10004,1.750000
...,...,...
17612,99829,2.000000
17613,99833,2.000000
17614,99835,2.000000
17615,99901,1.444444


In [140]:
dfg_rest_prices_by_zip_df = dfg_rest_prices_by_zip.to_frame()

In [141]:
# Subida del dataset a GCS
destino = 'dfg_rest_prices_by_zip.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

# dfg_rest_prices_by_zip_df.to_parquet('dfg_rest_prices_by_zip.parquet')

Archivo temp.parquet subido a dfg_rest_prices_by_zip.parquet.


#### Creación de dataset de coordenadas

In [142]:
dfg_rest_coord = dfg_rest[['gmap_id', 'latitude', 'longitude', 'name', 'state', 'city', 'postal_code',]]
dfg_rest_coord = dfg_rest_coord.rename(columns={'gmap_id': 'business_id'})
dfg_rest_coord['source'] = 'google'

print(dfg_rest_coord.info())
dfg_rest_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  223485 non-null  object 
 1   latitude     223485 non-null  float64
 2   longitude    223485 non-null  float64
 3   name         223485 non-null  object 
 4   state        223485 non-null  object 
 5   city         223485 non-null  object 
 6   postal_code  223485 non-null  object 
 7   source       223485 non-null  object 
dtypes: float64(2), object(6)
memory usage: 15.3+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source
1731186,0x89c2594f8f9984e3:0x54fd1d41e88d87a8,40.729347,-73.984240,Lhasa,NY,New York,10003,google
285072,0x89e45d9ac049228f:0x379c7564f5f7c659,41.939572,-71.348031,Pokemoto,MA,North Attleborough,2760,google


In [143]:
# Función para convertir latitud y longitud a coordenadas cartesianas
def lat_lon_to_cartesian(lat, lon):
    R = 6371  # Radio de la Tierra en kilómetros
    x = R * math.cos(math.radians(lat)) * math.cos(math.radians(lon))
    y = R * math.cos(math.radians(lat)) * math.sin(math.radians(lon))
    return x, y

# Suponiendo que tienes un DataFrame llamado df con las columnas business_id, latitud y longitud
# Agrega campos de coordenadas cartesianas x e y al DataFrame
dfg_rest_coord['x'], dfg_rest_coord['y'] = zip(*dfg_rest_coord.apply(lambda row: lat_lon_to_cartesian(row['latitude'], row['longitude']), axis=1))

print(dfg_rest_coord.info())
dfg_rest_coord.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 3025006
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  223485 non-null  object 
 1   latitude     223485 non-null  float64
 2   longitude    223485 non-null  float64
 3   name         223485 non-null  object 
 4   state        223485 non-null  object 
 5   city         223485 non-null  object 
 6   postal_code  223485 non-null  object 
 7   source       223485 non-null  object 
 8   x            223485 non-null  float64
 9   y            223485 non-null  float64
dtypes: float64(4), object(6)
memory usage: 18.8+ MB
None


,business_id,latitude,longitude,name,state,city,postal_code,source,x,y
1820898,0x8085809086f67c01:0xf6ae91482c965482,37.783248,-122.415668,Mộng Thu Cafe,CA,San Francisco,94102,google,-2699.167727,-4250.638088
1613827,0x89b7a123440b5849:0x945fab2a05702226,38.632641,-76.899925,La Hacienda 2,MD,Waldorf,20601,google,1128.004958,-4847.283852


In [144]:
# Subida del dataset a GCS
destino = 'dfg_rest_coord.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

# Exportar
#dfg_rest_coord.to_parquet('dfg_rest_coord.parquet', index=False)
#dfg_rest_coord.to_excel('dfg_rest_coord.xlsx', index=False)

Archivo temp.parquet subido a dfg_rest_coord.parquet.


### Dataset REVIEW

In [5]:
dfg_rest = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_rest.parquet")

dfg_rest_ids = dfg_rest[['gmap_id', 'state']]

print(dfg_rest_ids.info())
dfg_rest_ids.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 248852 entries, 2 to 3025006
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  248852 non-null  object
 1   state    221749 non-null  object
dtypes: object(2)
memory usage: 5.7+ MB
None


,gmap_id,state
2690910,0x87d8cd57e918db7f:0x7c4d0340d0685691,None
1791413,0x88f8a943a7176b1f:0x9ac19a8fd5f3ec33,Permanently closed


In [6]:
# Lista para almacenar los dataframes filtrados
dataframes_filtrados_rest = []

# Función para procesar cada archivo JSON
def procesar_archivo_json(filepath):
    try:
        df = pd.read_json(filepath, lines=True)
        # Filtrar las filas según los gmap_ids deseados
        df_filtrado = df[df['gmap_id'].isin(dfg_rest_ids["gmap_id"])]
        dataframes_filtrados_rest.append(df_filtrado)
    except Exception as e:
        print(f"Error al procesar el archivo {filepath}: {e}")

In [156]:
# Carga de datos desde los datasets .json para cada estado de USA

#states = [
#    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
#    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
#    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
#    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
#    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
#    'District_of_Columbia'
#]
#df_list = []

#for state in states:
#    filename = f'/Users/Juan/Documents/Academics/DataScience/PjFinal/Google/GoogleReviews/revgoo_{state}.parquet'
#    dfgrev = pd.read_parquet(filename)
#    dfgrev = dfgrev.dropna(subset=['text'])

    # Filtrado por restaurantes
#    dfgrev = dfgrev[dfgrev['gmap_id'].isin(dfg_rest_ids['gmap_id'])]
    # Análisis de sentimientos con VADER
#    analyzer = SentimentIntensityAnalyzer()
#    dfgrev['vader_polarity'] = dfgrev['text'].apply(lambda text: analyzer.polarity_scores(text)['compound'])
#    dfgrev['vader_sentiment'] = pd.cut(dfgrev['vader_polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])
    # Seleccion de campos
#    dfgrev = dfgrev[['gmap_id', 'state', 'user_id', 'time', 'rating', 'vader_polarity', 'vader_sentiment']]

#    df_list.append(dfgrev)
#dfg_reviews = pd.concat(df_list, ignore_index=True)

# print(dfg_reviews.shape)
# dfg_reviews.sample(2)

(7162236, 8)


,user_id,name,time,rating,text,gmap_id,vader_polarity,vader_sentiment
336158,1.089925e+20,Keaton Weinbrenner,1606236458720,4,good service and good pizza,0x87d28d5aa640a9c5:0x3c9cc5d3a99de80c,0.7003,1
2822320,1.035633e+20,Jon Swedberg,1531232260236,1,None,0x89e47d7f337876d7:0xbc34ba9e5ea67023,0.0000,0


In [7]:
# Diccionario de estados y rutas
estados_ruta = {
    "AL": "../Data/Google/reviews-estados/review-Alabama/review-Alabama",
    'AK':'../Data/Google/reviews-estados/review-Alaska/review-Alaska',
    'AZ':'../Data/Google/reviews-estados/review-Arizona/review-Arizona',
    'AR':'../Data/Google/reviews-estados/review-Arkansas/review-Arkansas',
    'CA':'../Data/Google/reviews-estados/review-California/review-California',
    'CO':'../Data/Google/reviews-estados/review-Colorado/review-Colorado',
    'CT':'../Data/Google/reviews-estados/review-Connecticut/review-Connecticut',
    'DE':"../Data/Google/reviews-estados/review-Delaware/review-Delaware",
    'District_of_Columbia':"../Data/Google/reviews-estados/review-District_of_Columbia",
    'FL':"../Data/Google/reviews-estados/review-Florida/review-Florida",
    'GA':"../Data/Google/reviews-estados/review-Georgia/review-Georgia",
    'HI':"../Data/Google/reviews-estados/review-Hawaii/review-Hawaii",
    'ID':"../Data/Google/reviews-estados/review-Idaho/review-Idaho",
    'IL':"../Data/Google/reviews-estados/review-Illinois/review-Illinois",
    'IN':"../Data/Google/reviews-estados/review-Indiana/review-Indiana",
    'IA':"../Data/Google/reviews-estados/review-Iowa/review-Iowa",
    'KS':"../Data/Google/reviews-estados/review-Kansas/review-Kansas",
    'KY':"../Data/Google/reviews-estados/review-Kentucky/review-Kentucky",
    'LA':"../Data/Google/reviews-estados/review-Louisiana/review-Louisiana",
    'ME':"../Data/Google/reviews-estados/review-Maine/review-Maine",
    'MD':"../Data/Google/reviews-estados/review-Maryland/review-Maryland",
    'MA':"../Data/Google/reviews-estados/review-Massachusetts/review-Massachusetts",
    'MI':"../Data/Google/reviews-estados/review-Michigan/review-Michigan",
    'MN':"../Data/Google/reviews-estados/review-Minnesota/review-Minnesota",
    'MS':"../Data/Google/reviews-estados/review-Mississippi/review-Mississippi",
    'MO':"../Data/Google/reviews-estados/review-Missouri/review-Missouri",
    'MT':"../Data/Google/reviews-estados/review-Montana/review-Montana",
    'NE':"../Data/Google/reviews-estados/review-Nebraska/review-Nebraska",
    'NV':"../Data/Google/reviews-estados/review-Nevada/review-Nevada",
    'NH':"../Data/Google/reviews-estados/review-New_Hampshire/review-New_Hampshire",
    'NJ':"../Data/Google/reviews-estados/review-New_Jersey/review-New_Jersey",
    'NM':"../Data/Google/reviews-estados/review-New_Mexico/review-New_Mexico",
    'NY':"../Data/Google/reviews-estados/review-New_York/review-New_York",
    'NC':"../Data/Google/reviews-estados/review-North_Carolina/review-North_Carolina",
    'ND':"../Data/Google/reviews-estados/review-North_Dakota/review-North_Dakota",
    'OH':"../Data/Google/reviews-estados/review-Ohio/review-Ohio",
    'OK':"../Data/Google/reviews-estados/review-Oklahoma/review-Oklahoma",
    'OR':"../Data/Google/reviews-estados/review-Oregon/review-Oregon",
    'PA':"../Data/Google/reviews-estados/review-Pennsylvania/review-Pennsylvania",
    'RI':"../Data/Google/reviews-estados/review-Rhode_Island/review-Rhode_Island",
    'SC':"../Data/Google/reviews-estados/review-South_Carolina/review-South_Carolina",
    'SD':"../Data/Google/reviews-estados/review-South_Dakota/review-South_Dakota",
    'TN':"../Data/Google/reviews-estados/review-Tennessee/review-Tennessee",
    'TX':"../Data/Google/reviews-estados/review-Texas/review-Texas",
    'UT':"../Data/Google/reviews-estados/review-Utah/review-Utah",
    'VT':"../Data/Google/reviews-estados/review-Vermont/review-Vermont",
    'VA':"../Data/Google/reviews-estados/review-Virginia/review-Virginia",
    'WA':"../Data/Google/reviews-estados/review-Washington/review-Washington",
    'WV':"../Data/Google/reviews-estados/review-West_Virginia/review-West_Virginia",
    'WI':"../Data/Google/reviews-estados/review-Wisconsin/review-Wisconsin",
    'WY':"../Data/Google/reviews-estados/review-Wyoming/review-Wyoming"
}

# Iterar sobre el diccionario y procesar los archivos
for state, ruta_directorio in estados_ruta.items():
    # Procesar cada archivo en el directorio
    for filename in os.listdir(ruta_directorio):
        if filename.endswith('.json'):
            filepath = os.path.join(ruta_directorio, filename)
            procesar_archivo_json(filepath)


# Concatenar todos los dataframes en uno solo
dfg_reviews = pd.concat(dataframes_filtrados_rest, ignore_index=True)

In [8]:
# Se eliminan los campos de pics y resp.
dfg_reviews.drop(columns=["pics","resp"],inplace=True)

In [9]:
dfg_reviews.head()

,user_id,name,time,rating,text,gmap_id
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a
1,1.143165e+20,Jonathan Robert,1447623939865,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a
2,1.136326e+20,Brian Harvey,1469293549247,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a
3,1.076840e+20,Ashley Maddox,1442279219480,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a
4,1.126036e+20,West Martin,1382634896130,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a


In [10]:
# preprocesado reviews 
dfg_reviews.to_parquet("../Data/Google/reviews-estados-parquet/dfg_reviews_preprocesados.parquet")

In [15]:
dfg_reviews = pd.read_parquet("../Data/Google/reviews-estados-parquet/dfg_reviews_preprocesados.parquet")
dfg_reviews.head()

,user_id,name,time,rating,text,gmap_id
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a
1,1.143165e+20,Jonathan Robert,1447623939865,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a
2,1.136326e+20,Brian Harvey,1469293549247,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a
3,1.076840e+20,Ashley Maddox,1442279219480,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a
4,1.126036e+20,West Martin,1382634896130,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a


In [19]:
dfg_reviews_usa = dfg_reviews

In [20]:
dfg_reviews_usa['time'] = dfg_reviews_usa['time'] / 1000

# Convertir la marca de tiempo a un objeto de fecha y hora
dfg_reviews_usa['datetime'] = dfg_reviews_usa['time'].apply(lambda x: datetime.fromtimestamp(x))

dfg_reviews_usa.head()

,user_id,name,time,rating,text,gmap_id,datetime
0,1.179759e+20,Anthony Roberts,1.463443e+06,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 17:30:43.013514
1,1.143165e+20,Jonathan Robert,1.447624e+06,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 13:07:03.939865
2,1.136326e+20,Brian Harvey,1.469294e+06,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 19:08:13.549247
3,1.076840e+20,Ashley Maddox,1.442279e+06,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 11:37:59.219480
4,1.126036e+20,West Martin,1.382635e+06,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-16 19:03:54.896130


In [21]:
dfg_reviews_sent = dfg_reviews_usa

# Análisis de sentimientos con VADER
analyzer = SentimentIntensityAnalyzer()

# Aplicar análisis de sentimientos, manejando valores nulos
dfg_reviews_sent['vader_polarity'] = dfg_reviews_sent['text'].apply(
    lambda text: analyzer.polarity_scores(text)['compound'] if pd.notnull(text) else 0
)

# Clasificar sentimientos según la polaridad
dfg_reviews_sent['vader_sentiment'] = pd.cut(
    dfg_reviews_sent['vader_polarity'], 
    bins=[-float('inf'), -0.001, 0.0, float('inf')], 
    labels=[-1, 0, 1]
)  

dfg_reviews_sent.head()

,user_id,name,time,rating,text,gmap_id,datetime,vader_polarity,vader_sentiment
0,1.179759e+20,Anthony Roberts,1.463443e+06,4,"On the higher end of price for pizza, but they...",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 17:30:43.013514,0.3898,1
1,1.143165e+20,Jonathan Robert,1.447624e+06,3,"Food was ok, felt like the atmosphere as well ...",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 13:07:03.939865,0.6275,1
2,1.136326e+20,Brian Harvey,1.469294e+06,4,"Good food, service so so",0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 19:08:13.549247,0.4404,1
3,1.076840e+20,Ashley Maddox,1.442279e+06,5,Love it,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 11:37:59.219480,0.6369,1
4,1.126036e+20,West Martin,1.382635e+06,5,Yum,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-16 19:03:54.896130,0.0000,0
...,...,...,...,...,...,...,...,...,...
12948984,1.122427e+20,Fallon Busby,1.518055e+06,5,None,0x5335fabe642bd1c9:0x5802befac5636710,1970-01-18 08:40:54.560572,0.0000,0
12948985,1.097046e+20,Jaiegh Huutch,1.582433e+06,4,None,0x5335fabe642bd1c9:0x5802befac5636710,1970-01-19 02:33:53.438594,0.0000,0
12948986,1.117050e+20,Ryan Jarvis,1.592165e+06,4,None,0x5335fabe642bd1c9:0x5802befac5636710,1970-01-19 05:16:05.160866,0.0000,0
12948987,1.090081e+20,Christian McCune,1.554232e+06,4,None,0x5335fabe642bd1c9:0x5802befac5636710,1970-01-18 18:43:51.641410,0.0000,0


In [23]:
# Se obtiene el dataset final con las columnas requeridas.
dfg_reviews = dfg_reviews_usa[['user_id', 'gmap_id', 'datetime', 'rating', 'vader_polarity', 'vader_sentiment']]
dfg_reviews.head()

,user_id,gmap_id,datetime,rating,vader_polarity,vader_sentiment
0,1.179759e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 17:30:43.013514,4,0.3898,1
1,1.143165e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 13:07:03.939865,3,0.6275,1
2,1.136326e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 19:08:13.549247,4,0.4404,1
3,1.076840e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-17 11:37:59.219480,5,0.6369,1
4,1.126036e+20,0x8889221157fb3455:0x5c125c40c3eccc2a,1970-01-16 19:03:54.896130,5,0.0000,0


In [24]:
print(dfg_reviews.info())
dfg_reviews.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948989 entries, 0 to 12948988
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   user_id          float64       
 1   gmap_id          object        
 2   datetime         datetime64[ns]
 3   rating           int64         
 4   vader_polarity   float64       
 5   vader_sentiment  category      
dtypes: category(1), datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 506.3+ MB
None


,user_id,gmap_id,datetime,rating,vader_polarity,vader_sentiment
8282821,1.148675e+20,0x89c259a8f8550c75:0x24b62f76d52d20b3,1970-01-18 07:25:32.756858,1,0.9836,1
588779,1.159526e+20,0x87d428cff3f3adbb:0x77119ffc33f8fdd4,1970-01-19 10:54:01.994333,5,0.4215,1
11992940,1.161238e+20,0x89babb1ad941e53f:0x3c70d8eaeb7b3087,1970-01-19 02:12:47.553890,5,0.0000,0
1166560,1.127315e+20,0x80845698b7c6cf11:0x745f1bcf2fee460d,1970-01-18 14:53:27.646757,5,-0.7620,-1
8699198,1.088531e+20,0x89ace8b7c32115f9:0x7bd48fa099ce1e3c,1970-01-18 14:51:22.296407,4,0.0000,0


In [25]:
# Subida del dataset a GCS
destino = 'dfg_reviews.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

# Exportación de datos
#dfg_reviews_usa.to_parquet('dfgrevall.parquet')

Archivo temp.parquet subido a dfg_reviews.parquet.


## Conclusiones

* Dataset SITES
    * Conversión del campo "precio" a formato numérico
    * Obtención de los campos "state" y "postal_code" a partir del campo "address" - esto será fundamental para combinar luego el dataset de google y yelp
    * Creación del dataset de categorías a partir del campo "category" cuyos valores originales están en formato tipo "lista" - será de utilidad para el modelo de machine learning
    * Creación del dataset de atributos a partir del campo "MISC" cuyos valores originales están en formato tipo "diccionario de listas" - será de utilidad para el modelo de machine learning
    * Creación del dataset de precios - será utilizado para crear el campo "precio" en el dataset de yelp
    * Creación del dataset de coordenadas - se utilizará para concatenar un listado único de sites entre ambos datasets de google y yelp
* Dataset REVIEWS
    * Mediante un código se leen todos los archivos de reviews de cada estado, concatenando los registros que correspondan a restaurantes

# ETL Yelp - 1

## Carga de datos
=======================================================================================

In [26]:
# Yelp
business = pd.read_pickle("../Data/Yelp/business.pkl")
checkin = pd.read_json('../Data/Yelp/checkin.json', lines=True)
user = pd.read_parquet('../Data/Yelp/user.parquet')

ArrowMemoryError: realloc of size 2948595712 failed

In [ ]:
# Base de datos externa
df_uszip = pd.read_excel('dataset_e_uszips.xlsx')

## Preprocesamiento
====================================================================================

### Dataset uszip (externo)

In [ ]:
print(df_uszip.info())
df_uszip.sample(2)

In [ ]:
df_uszip = df_uszip[['postal_code', 'state']]
df_uszip['postal_code'] = df_uszip['postal_code'].astype(str)

df_uszip.info()
df_uszip.sample(2)

### Dataset BUSINESS

In [ ]:
print(business.info())
business.sample(5)

In [ ]:
business_copy = business.copy()
business_copy = business_copy.loc[:,~business_copy.columns.duplicated()]
business_copy.head()

In [ ]:
# Selección de campos
dfybsn = business[['business_id', 'name', 'state', 'city', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open']]

# Eliminación de columnas duplicadas
dfybsn = dfybsn.loc[:,~dfybsn.columns.duplicated()]

# Adcuación de tipos de datos
dfybsn['latitude'] = pd.to_numeric(dfybsn['latitude'], errors='coerce')
dfybsn['longitude'] = pd.to_numeric(dfybsn['longitude'], errors='coerce')
dfybsn['stars'] = pd.to_numeric(dfybsn['stars'], errors='coerce')
dfybsn['review_count'] = pd.to_numeric(dfybsn['review_count'], errors='coerce')
dfybsn['is_open'] = pd.to_numeric(dfybsn['is_open'], errors='coerce')

# Eliminación de duplicados
dfybsn.drop_duplicates()

'''Reemplazo de valores del campo "state": los valores del dataset original no se corresponden con los códigos postales, 
por lo cual se utilizará una base de datos externa para corregir esta serie.'''

dfybsn.drop(columns=['state'], inplace=True)
dfybsn = pd.merge(dfybsn, df_uszip, on='postal_code', how='left')

# Campos auxiliares
dfybsn.dropna(subset=['state'], inplace=True)
dfybsn['state_city'] = dfybsn['state'].str.cat(dfybsn['city'], sep=' - ')
dfybsn['city_postalcode'] = dfybsn['city'].str.cat(dfybsn['postal_code'], sep=' - ')
dfybsn['state_city_postalcode'] = dfybsn['state'].str.cat(dfybsn['city'], sep=' - ').str.cat(dfybsn['postal_code'], sep=' - ')

dfybsn.info()
dfybsn.sample(2)

In [ ]:
# Subida del dataset a GCS
destino = 'dfg_reviews.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

### Creación de dataset RESTAURANTES

In [ ]:
# Selección de campos
dfybct = business[['business_id', 'categories']]

# Eliminación de columnas duplicadas
dfybct = dfybct.loc[:,~dfybct.columns.duplicated()]

# Adecuación de campos
dfybct = dfybct.explode('categories')
dfybct = dfybct.assign(categories=dfybct['categories'].str.split(', ')).explode('categories')
dfybct.reset_index(drop=True, inplace=True)

# Eliminación de nulos
dfybct = dfybct.dropna(subset=['categories'])

dfybct.info()
dfybct.sample(2)

In [ ]:
# CATEGORIAS DISPONIBLES
#categories_yelp = pd.DataFrame(dfybct['categories'].unique())
#categories_yelp.to_csv('categories_yelp.csv', index=False)

In [ ]:
# CATEGORIAS TIPO "FOOD" (LOCALES QUE TIENEN POR LO MENOS UNA CATEGORIA GASTRONOMICA)
categories_food = pd.read_csv('dataframe_categories_food.csv')
dfyfct = pd.merge(dfybct, categories_food, on='categories', how='inner')

dfyfct.info()
dfyfct.sample(2)

In [ ]:
dfy_site_categories = dfyfct.copy()
dfy_site_categories.rename(columns={"business_id":"site_id"}, inplace=True)
dfy_site_categories.to_parquet("dfy_site_categories.parquet")

In [ ]:
dfyfct['categories'].nunique()

In [ ]:
# LOCALES QUE SON RESTAURANTE
dfyrst = pd.merge(dfybsn, dfyfct, on='business_id', how='left')
# Eliminación de duplicados
#dfyrst = dfyrst.drop_duplicates(subset='business_id', keep='first')

dfyrst.info()
dfyrst.sample(2)

In [ ]:
# Agregado de campo "precio", basado en el dataset de google
dfg_rest_prices_by_zip = pd.read_parquet("dfg_rest_prices_by_zip.parquet")
dfg_rest_prices_by_zip.sample(5)
dfyrst = pd.merge(dfyrst, dfg_rest_prices_by_zip, on=['postal_code'], how='left')

dfyrst.info()
dfyrst.sample(2)

In [ ]:
# Subida del dataset a GCS
destino = 'dfg_reviews.parquet'
dataframe_to_parquet_and_upload(dfg_rest,bucket_name,destino)

# dfyrst.to_parquet('dfy_rest.parquet')

### Creación de dataset de coordenadas

In [ ]:
dfyrst_coord = dfyrst[['business_id', 'latitude', 'longitude', 'name', 'state', 'city', 'postal_code']]

dfyrst_coord['source'] = 'yelp'
dfyrst_coord.sample(2)

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

In [ ]:
# Función para convertir latitud y longitud a coordenadas cartesianas
def lat_lon_to_cartesian(lat, lon):
    R = 6371  # Radio de la Tierra en kilómetros
    x = R * math.cos(math.radians(lat)) * math.cos(math.radians(lon))
    y = R * math.cos(math.radians(lat)) * math.sin(math.radians(lon))
    return x, y

# Suponiendo que tienes un DataFrame llamado df con las columnas business_id, latitud y longitud
# Agrega campos de coordenadas cartesianas x e y al DataFrame
dfyrst_coord['x'], dfyrst_coord['y'] = zip(*dfyrst_coord.apply(lambda row: lat_lon_to_cartesian(row['latitude'], row['longitude']), axis=1))

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

In [ ]:
# Exportar 
#dfyrst_coord.to_parquet('dfyrst_coord.parquet', index=False)
#dfyrst_coord.to_excel('dfyrst_coord.xlsx', index=False)

### Creación de dataset BUSINESS ATTRIBUTES

In [ ]:
# Selección de campos
dfybat = business[['business_id','attributes']]

# Eliminación de columnas duplicadas
dfybat = dfybat.loc[:,~dfybat.columns.duplicated()]

dfybat.info()
dfybat.sample(5)

In [ ]:
# Adecuación de campos
dfybat = dfybat.explode('attributes')

# Eliminación de duplicados
dfybat.drop_duplicates()

# Eliminación de nulos
dfybat = dfybat.dropna(subset=['attributes'])

In [ ]:
dfybat.info()
dfybat.sample(5)

In [ ]:
#dfybat[['business_id', 'attributes']].to_parquet('dfy_attributes.parquet')
#dfybat[['business_id', 'attributes']].to_csv('dfy_attributes.csv')

### Precios

In [ ]:
'''# Filtrar los valores del campo "attributes" relacionados con la palabra "precio"
df_prices = dfybat[dfybat['attributes'].str.contains('price', case=False)]
df_prices.rename(columns={'attributes': 'price'}, inplace=True)

print(df_prices['price'].nunique())
# Mostrar el DataFrame filtrado
df_prices.sample(2)'''
# Se ha encontrado que este campo no contiene información de precios ni de rangos de precios

### Dataset CHECKIN

In [ ]:
print(checkin.shape)
checkin.head(2)

In [ ]:
# Filtrado por restarurants y agregado de características
dfychk = pd.merge(checkin, dfyrst, on='business_id', how='inner')

dfychk.info()
dfychk.head(2)

In [ ]:
dfychk['categories'].nunique()

In [ ]:
#dfychk = dfychk.assign(date=dfychk['date'].str.split(', ')).explode('date')

In [ ]:
dfychk['date'] = pd.to_datetime(dfychk['date'], errors='coerce')
dfychk['year'] = dfychk['date'].dt.year
dfychk['month'] = dfychk['date'].dt.month

# Filtrado cronológico (los datos están entre dic2009 y ene2022, por lo tanto se quitan los años 2009 y 2022)
dfychk = dfychk[(dfychk['year'] >= 2010) & (dfychk['year'] <= 2021)]

dfychk.info()
dfychk.sample(2)

In [ ]:
#dfychk.to_parquet('dfy_checkins.parquet')

### Dataset USER

In [ ]:
user.info()
user.sample(2)

In [ ]:
# Selección de campos
dfyusr = user[['user_id', 'review_count', 'fans', 'yelping_since', 'useful', 'funny', 'cool', 'average_stars']]

dfyusr['yelping_since'] = pd.to_datetime(dfyusr['yelping_since'], errors='coerce')
dfyusr.reset_index(drop=True, inplace=True)

dfyusr['year'] = dfyusr['yelping_since'].dt.year
dfyusr['month'] = dfyusr['yelping_since'].dt.month
dfyusr['year_month'] = dfyusr['year'].astype(str).str.slice(-2) + dfyusr['month'].astype(str).str.zfill(2)

dfyusr.info()
dfyusr.sample(5)

In [ ]:
dfyusr.to_parquet("dfy_user.parquet")

## Conclusiones

* Dataset BUSINESS
   * Se corrigen los códigos postales originales pues estaban erróneos
   * Se eliminan columnas duplicadas
   * Se adecúan los tipos de datos para su posterior procesamiento
   * Se eliminan registros duplicados
* Creación de dataset RESTAURANTES
    Mediante un archivo que contiene las categorías deseadas de restaurantes y bares, se filtran los locales a considerar
    Se agrega el campo precio tomado del dataset de google, con el cual se estima el rango de precios del local tomando el promedio del código postal al que pertenece
* Creación del dataset de COORDENADAS
    * Se calculan las coordenadas cartesianas y se exporta un archivo para luego crear un dataset de locales unívocos tomando en conjunto a yelp y google
* Creación del dataset ATRIBUTOS
    * Explotando el campo "attributes" se exporta un dataset con los atributos que se consideran relevantes para el modelo de machine learning
* Dataset CHECKIN
    * Se obtiene este registro de visitas mediante el filtrado de locales tipo restaurants & bares
* Dataset USER
    * Se adecúan los tipos de datos para su posterior procesamiento

# ETL YELP - 2

## Carga de datos
=============================================================

In [ ]:
# Restaurantes
dfyrst = pd.read_parquet('dfyrst_gastronomics.parquet')

In [ ]:
# Tips
tip = pd.read_json('dataset_y_tips.json', lines=True)

In [ ]:
# Review
review = pd.read_parquet('dataset_y_reviews.parquet')

## Preprocesamiento
=========================================================================

### Dataset RESTAURANTES

In [ ]:
dfyrst.sample()

### Dataset TIP

In [ ]:
tip.sample(2)

In [ ]:
# Selección de campos
dfytip = tip

dfytip['year'] = dfytip['date'].dt.year
dfytip['month'] = dfytip['date'].dt.month
dfytip['year_month'] = dfytip['year'].astype(str).str.slice(-2) + dfytip['month'].astype(str).str.zfill(2)

# Filtrado por restaurantes
dfytip = dfytip[dfytip['business_id'].isin(dfyrst['business_id'])]
dfytip

dfytip.info()
dfytip.sample(5)

In [ ]:
# Análisis de sentimientos a partir del campo "text"
dfytip['text'] = dfytip['text'].astype(str)

analyzer = SentimentIntensityAnalyzer()
dfytip['polarity'] = dfytip['text'].apply(lambda text: analyzer.polarity_scores(text)['compound'])
dfytip['sentiment'] = pd.cut(dfytip['polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])

dfytip.info()
dfytip.sample(5)

In [ ]:
dfytip.to_parquet('dfy_tips.parquet', index=False)

### Dataset REVIEW

In [ ]:
print(review.shape)
review.sample(2)

In [ ]:
# Selección de campos
dfyrev = review
# Adecuación de campos
dfyrev['date'] = pd.to_datetime(dfyrev['date'], errors='coerce')
dfyrev.reset_index(drop=True, inplace=True)

# Eliminación de duplicados: No tiene duplicados, se ha analizado fuera de este archivo

# Eliminación de nulos
dfyrev = dfyrev.dropna()

dfyrev['year'] = dfyrev['date'].dt.year
dfyrev['month'] = dfyrev['date'].dt.month
dfyrev['year_month'] = dfyrev['year'].astype(str).str.slice(-2) + dfyrev['month'].astype(str).str.zfill(2)

dfyrev = dfyrev[(dfyrev['year'] >= 2010) & (dfyrev['year'] <= 2021)]

# Filtrado por restaurantes
dfyrev = dfyrev[dfyrev['business_id'].isin(dfyrst['business_id'])]
dfyrev

dfyrev.info()
dfyrev.sample(2)

In [ ]:
# Especificar la semilla aleatoria para reproducibilidad
random_state = 42
dfyrev_sample = dfyrev.sample(n=1000000, random_state=random_state)
dfyrev_sample.head(2)

In [ ]:
analyzer = SentimentIntensityAnalyzer()

dfyrev_sample['polarity'] = [analyzer.polarity_scores(text)['compound'] for text in dfyrev_sample['text']]
dfyrev_sample['sentiment'] = pd.cut(dfyrev_sample['polarity'], bins=[-float('inf'), -0.001, 0.0, float('inf')], labels=[-1, 0, 1])

print(dfyrev_sample.info())
dfyrev_sample.sample(2)

## Conclusiones
==========================================================================================================================================00

* El dataset RESTAURANTES se encuentra ya procesado y se utilizará para filtrar los locales deseados utilizando el campo 'site_id"
* Dataset TIP
    * Contiene información de reseñas con un texto un poco más corto pero con sugerencias más rápidas
    * Se adecúan los tipos de datos, no se observan valores nulos ni duplicados
    * Se aplica un análisis de sentimientos utilizando "vader"
* Dataset REVIEWS
    * Se adecúan los tipos de datos, no se observan valores nulos, se eliminan los pocos duplicados que tiene
    * Se aplica un análisis de sentimientos utilizando "vader" tomando una muestra de 1 millón de registros (contiene aprox 6 millones)

# ETL Unicos

## Contenido
El presente archivo se realiza con la intención de buscar sites repetidos en los datasets, con el objetivo de crear un listado de locales únicos para luego unir ambos datasets.

El plan es el siguiente: se conservarán todos los locales del dataset de GOOGLE, y se eliminarán los locales que se encuentren repetidos en el dataset de YELP.

## Carga de datos
===========================================================================================================================000

In [ ]:
# Lectura de dataset de restaurants de GOOGLE
dfgrst_coord = pd.read_parquet('dfgrst_coord.parquet')

dfgrst_coord['x'] = dfgrst_coord['x'].round(2)
dfgrst_coord['y'] = dfgrst_coord['y'].round(2)

dfgrst_coord['latitude'] = dfgrst_coord['latitude'].round(4)
dfgrst_coord['longitude'] = dfgrst_coord['longitude'].round(4)

print(dfgrst_coord.info())
dfgrst_coord.sample(2)

In [ ]:
# Lectura de dataset de restaurants de YELP
dfyrst_coord = pd.read_parquet('dfyrst_coord.parquet')

dfyrst_coord['x'] = dfyrst_coord['x'].round(2)
dfyrst_coord['y'] = dfyrst_coord['y'].round(2)
dfyrst_coord['postal_code'] = dfyrst_coord['postal_code'].astype('int64')

dfyrst_coord['latitude'] = dfyrst_coord['latitude'].round(4)
dfyrst_coord['longitude'] = dfyrst_coord['longitude'].round(4)

print(dfyrst_coord.info())
dfyrst_coord.sample(2)

## Procesamiento
==============================================================================================================

In [ ]:
# Locales repetidos
df_gy = pd.merge(dfgrst_coord, dfyrst_coord, how='inner', on=['postal_code', 'x', 'y'])
print(df_gy.info())
df_gy.sample(2)

In [ ]:
# Locales a eliminar en dfy
dfy_a_eliminar = df_gy['business_id_y']
dfy_a_eliminar = dfy_a_eliminar.to_frame()

print(dfy_a_eliminar.info())
dfy_a_eliminar.sample(2)

In [ ]:
# Listado final de locales a conservar en dfyrst (restaurantes de YELP)
dfy_rst_final = dfyrst_coord[~dfyrst_coord['business_id'].isin(dfy_a_eliminar['business_id_y'])]

print(dfy_rst_final.info())
dfy_rst_final.sample(2)

In [ ]:
# Listado final de locales a conservar en dfgyrst (restaurantes de YELP y GOOGLE en conjunto)
dfgy_rst_final = pd.concat([dfgrst_coord, dfy_rst_final])

print(dfgy_rst_final.info())
dfgy_rst_final.sample(2)

### Exportación de datos

In [1]:
# Nota: el cógido se deja comentado a propósito para que al correr el código se evite la sobreescritura 
# de los archivos ya generados

#dfgy_rst_final.to_parquet('dfgy_rest_uniques.parquet')
#dfy_rst_final.to_parquet('dfy_rest_uniques.parquet')

## Conclusiones

El objetivo de este ETL es lograr un listado único de locales, uniendo ambos datasets de yelp y google.

* Se conserva el campo "site_id" original de cada dataset, pero como cada dataset tiene un formatto diferente, se busca otra alternativa para identificar en forma unívoca a cada site
* Para hallar locales coincidentes, se utilizan los campos de coordenadas cartesianas "x" e "y", y además el campo "postal_code"
* Finalmente, se conserva el listado original de google, y se eliminan los locales repetidos en el listado de yelp

# Unión final de datasets

## Carga de datos
===============================================================================================================================000

### Unicos

In [4]:
# Listado unívoco de locales de ambos datasets
dfgy_rest_uniques = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfgy_rest_uniques.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\1129644781.py:2: SyntaxWarning: invalid escape sequence '\D'
  dfgy_rest_uniques = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfgy_rest_uniques.parquet')


### Yelp

In [6]:
# Yelp
yelp_rest = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_rest.parquet')
yelp_user = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_user.parquet')
yelp_checkins = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_checkins.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\1449867972.py:2: SyntaxWarning: invalid escape sequence '\D'
  yelp_rest = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_rest.parquet')
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\1449867972.py:3: SyntaxWarning: invalid escape sequence '\D'
  yelp_user = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_user.parquet')
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\1449867972.py:4: SyntaxWarning: invalid escape sequence '\D'
  yelp_checkins = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_checkins.parquet')


In [7]:
# Review
yelp_reviews = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_reviews.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\819623371.py:2: SyntaxWarning: invalid escape sequence '\D'
  yelp_reviews = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_reviews.parquet')


In [8]:
# Tips
yelp_tips = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_tips.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\2310033575.py:2: SyntaxWarning: invalid escape sequence '\D'
  yelp_tips = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_tips.parquet')


In [9]:
# Categorías y atributos
yelp_site_categories = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_site_categories.parquet')
yelp_site_attributes = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_attributes.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\3408893820.py:2: SyntaxWarning: invalid escape sequence '\D'
  yelp_site_categories = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_site_categories.parquet')
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\3408893820.py:3: SyntaxWarning: invalid escape sequence '\D'
  yelp_site_attributes = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfy_attributes.parquet')


### Google

In [10]:
# Restaurants
google_rest = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_rest.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\623300417.py:2: SyntaxWarning: invalid escape sequence '\D'
  google_rest = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_rest.parquet')


In [11]:
# Reviews
google_reviews = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_reviews.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\440844640.py:2: SyntaxWarning: invalid escape sequence '\D'
  google_reviews = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_reviews.parquet')


In [12]:
# Categorías y atributos
google_site_categories = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_site_categories.parquet')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\dmon2\AppData\Local\Temp\ipykernel_16872\19224335.py:2: SyntaxWarning: invalid escape sequence '\D'
  google_site_categories = pd.read_parquet('..\Data\dataframes_to_repo\EDA_df_to_repo\dfg_site_categories.parquet')


## Preprocesamiento
============================================================================================================0000

### Datasets de YELP

#### YELP reviews

In [13]:
print(yelp_reviews.info())
yelp_reviews.sample(2)
# Nota: el dataset constituye una muestra 1 millón de registros debido a que el dataset de reviews es mucho mayor. 
# A Efectos del EDA, se considera suficiente.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1000000 non-null  object        
 1   user_id      1000000 non-null  object        
 2   business_id  1000000 non-null  object        
 3   stars        1000000 non-null  float64       
 4   useful       1000000 non-null  int64         
 5   funny        1000000 non-null  int64         
 6   cool         1000000 non-null  int64         
 7   text         1000000 non-null  object        
 8   date         1000000 non-null  datetime64[ns]
 9   year         1000000 non-null  int32         
 10  month        1000000 non-null  int32         
 11  year_month   1000000 non-null  object        
 12  polarity     1000000 non-null  float64       
 13  sentiment    1000000 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int32(2), int64(4), object(5)
mem

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year,month,year_month,polarity,sentiment
637882,RqB4K0dJIIk_65PitROCOQ,snbV1KcikYo-y_49kjKv9A,9y9YOmix4f5xqtJi7_XgGQ,1.0,1,1,1,Some seriously rude staff working at this join...,2017-10-02 00:52:25,2017,10,1710,-0.2843,-1
552276,ubyvxs16MX2AqhEqh0ALzg,tlAb58AJssGiSiXJHWGGBQ,bncTqUdA8ZPcUkDDmUbqyA,4.0,1,0,0,Finally ! A gluten free menu at a brewery!\nTh...,2011-05-16 14:17:49,2011,5,1105,0.8856,1


In [14]:
dfy_reviews = yelp_reviews.copy()

dfy_reviews['source'] = 'yelp'
dfy_reviews = dfy_reviews[['source','business_id','user_id', 'date', 'month', 'year', 'stars', 'polarity', 'sentiment']]
dfy_reviews = dfy_reviews.rename(columns={'business_id': 'site_id', 'date':'datetime', 'stars':'rating'})

print(dfy_reviews.info())
dfy_reviews.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   source     1000000 non-null  object        
 1   site_id    1000000 non-null  object        
 2   user_id    1000000 non-null  object        
 3   datetime   1000000 non-null  datetime64[ns]
 4   month      1000000 non-null  int32         
 5   year       1000000 non-null  int32         
 6   rating     1000000 non-null  float64       
 7   polarity   1000000 non-null  float64       
 8   sentiment  1000000 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(3)
memory usage: 61.0+ MB
None


,source,site_id,user_id,datetime,month,year,rating,polarity,sentiment
482598,yelp,4f6LBuHB1G8owOOZgm5IEA,jMJOOe8Ix9EFcY8GY4b08g,2020-12-30 02:59:59,12,2020,5.0,0.8766,1
548848,yelp,5Ce3lZksYVkCbrihqylVHQ,WfietwxXBDs_whgmYKc49w,2014-05-07 12:29:13,5,2014,5.0,0.9656,1


#### YELP restaurants

In [15]:
print(yelp_rest.info())
yelp_rest.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212998 entries, 0 to 212997
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   business_id            212998 non-null  object 
 1   name                   212998 non-null  object 
 2   city                   212998 non-null  object 
 3   postal_code            212998 non-null  object 
 4   latitude               212998 non-null  float64
 5   longitude              212998 non-null  float64
 6   stars                  212998 non-null  float64
 7   review_count           212998 non-null  int64  
 8   is_open                212998 non-null  int64  
 9   state                  212998 non-null  object 
 10  state_city             212998 non-null  object 
 11  city_postalcode        212998 non-null  object 
 12  state_city_postalcode  212998 non-null  object 
 13  categories             133777 non-null  object 
 14  food                   133777 non-nu

,business_id,name,city,postal_code,latitude,longitude,stars,review_count,is_open,state,state_city,city_postalcode,state_city_postalcode,categories,food,price
168184,BG5ErcFzXJ0tG9rzvKtyjg,The Coves of Brighton Bay - Matrix Residential,St Petersburg,33716,27.871708,-82.624055,1.5,8,0,FL,FL - St Petersburg,St Petersburg - 33716,FL - St Petersburg - 33716,None,None,1.750000
200931,vjlMxdeqRKQCo7ZJ7BKxPw,Hard Rock Cafe Sales,Nashville,37201,36.162405,-86.774979,4.0,6,1,TN,TN - Nashville,Nashville - 37201,TN - Nashville - 37201,None,None,1.857143
137430,iXaVR134oU7VnxFPVPWjvg,B Willey Hair Studio,New Orleans,70119,29.977539,-90.100015,5.0,8,1,LA,LA - New Orleans,New Orleans - 70119,LA - New Orleans - 70119,None,None,1.466667
120260,SZZX4dKRKT2OQPAcBjjPNw,NOLA Super Buffet,Kenner,70065,30.027380,-90.238327,2.0,51,0,LA,LA - Kenner,Kenner - 70065,LA - Kenner - 70065,Buffets,yes,1.421053
184189,ykvRMbh0HYTq5UYQ5oExRg,Senor Burrito,Reno,89502,39.474247,-119.781529,4.0,7,0,NV,NV - Reno,Reno - 89502,NV - Reno - 89502,Fast Food,yes,1.333333


In [16]:
dfy_rest = yelp_rest

dfy_rest['source'] = 'yelp'
dfy_rest = dfy_rest[['source','business_id', 'name', 'state', 'city', 'postal_code', 'price', 'stars', 'review_count']]
dfy_rest = dfy_rest.rename(columns={'business_id': 'site_id', 'stars':'rating_avg', 'review_count':'reviews_count'})

dfy_rest.dropna(subset=['price'], inplace=True)

# Agregado de fecha de inicio de acitividad
dfy_rest_date_start = dfy_reviews.groupby('site_id')['datetime'].min().reset_index()
dfy_rest = pd.merge(dfy_rest, dfy_rest_date_start, how='left', on=['site_id'])
dfy_rest = dfy_rest.rename(columns={'datetime': 'date_start'})

dfy_rest['year'] = dfy_rest['date_start'].dt.year
dfy_rest['month'] = dfy_rest['date_start'].dt.month

dfy_rest['state_city'] = dfy_rest['state'].str.cat(dfy_rest['city'], sep=' - ')
dfy_rest['city_postalcode'] = dfy_rest['city'].str.cat(dfy_rest['postal_code'], sep=' - ')
dfy_rest['state_city_postalcode'] = dfy_rest['state'].str.cat(dfy_rest['city'], sep=' - ').str.cat(dfy_rest['postal_code'], sep=' - ')

print(dfy_rest.info())
dfy_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211084 entries, 0 to 211083
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   source                 211084 non-null  object        
 1   site_id                211084 non-null  object        
 2   name                   211084 non-null  object        
 3   state                  211084 non-null  object        
 4   city                   211084 non-null  object        
 5   postal_code            211084 non-null  object        
 6   price                  211084 non-null  float64       
 7   rating_avg             211084 non-null  float64       
 8   reviews_count          211084 non-null  int64         
 9   date_start             128792 non-null  datetime64[ns]
 10  year                   128792 non-null  float64       
 11  month                  128792 non-null  float64       
 12  state_city             211084 non-null  obje

,source,site_id,name,state,city,postal_code,price,rating_avg,reviews_count,date_start,year,month,state_city,city_postalcode,state_city_postalcode
28960,yelp,EWLgU-D60x6NM10AfJGUDg,Little Caesars Pizza,IN,Indianapolis,46254,1.300000,1.5,6,2021-12-20 01:20:20,2021.0,12.0,IN - Indianapolis,Indianapolis - 46254,IN - Indianapolis - 46254
64503,yelp,eDfmJgWv43ysajI3jdDIqw,Pizza World,IL,Granite City,62040,1.045455,3.0,12,2014-05-01 02:07:50,2014.0,5.0,IL - Granite City,Granite City - 62040,IL - Granite City - 62040


In [17]:
print(dfy_rest.info())
dfy_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211084 entries, 0 to 211083
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   source                 211084 non-null  object        
 1   site_id                211084 non-null  object        
 2   name                   211084 non-null  object        
 3   state                  211084 non-null  object        
 4   city                   211084 non-null  object        
 5   postal_code            211084 non-null  object        
 6   price                  211084 non-null  float64       
 7   rating_avg             211084 non-null  float64       
 8   reviews_count          211084 non-null  int64         
 9   date_start             128792 non-null  datetime64[ns]
 10  year                   128792 non-null  float64       
 11  month                  128792 non-null  float64       
 12  state_city             211084 non-null  obje

,source,site_id,name,state,city,postal_code,price,rating_avg,reviews_count,date_start,year,month,state_city,city_postalcode,state_city_postalcode
108501,yelp,lZbimwxtW5JPB6biT7_q3w,Carnitas y Taqueria La Frontera,IN,Indianapolis,46227,1.444444,5.0,14,2021-01-10 20:58:13,2021.0,1.0,IN - Indianapolis,Indianapolis - 46227,IN - Indianapolis - 46227
82177,yelp,lwja_rmdJH48WPFaV2662A,Big Lots,FL,Tarpon Springs,34689,1.375000,4.5,6,NaT,NaN,NaN,FL - Tarpon Springs,Tarpon Springs - 34689,FL - Tarpon Springs - 34689


#### YELP user

In [18]:
dfy_user =yelp_user.copy()
print(dfy_user.info())
dfy_user.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   review_count   int64         
 2   fans           int64         
 3   yelping_since  datetime64[ns]
 4   useful         int64         
 5   funny          int64         
 6   cool           int64         
 7   average_stars  float64       
 8   year           int32         
 9   month          int32         
 10  year_month     object        
dtypes: datetime64[ns](1), float64(1), int32(2), int64(5), object(2)
memory usage: 160.6+ MB
None


,user_id,review_count,fans,yelping_since,useful,funny,cool,average_stars,year,month,year_month
854328,HpwQOx-11JhMe5_H3jloQw,12,0,2016-12-31 01:07:48,3,2,0,3.58,2016,12,1612
764226,mgGk2F-JqM_TwgX90p53cQ,3,0,2013-12-11 05:06:39,4,0,1,5.00,2013,12,1312


In [19]:
# Selección de campos
dfy_user = dfy_user[['user_id', 'review_count', 'yelping_since', 'average_stars']]
dfy_user = dfy_user.rename(columns={'review_count':'reviews_count', 'yelping_since':'date_start', 'average_stars':'rating_avg'})

dfy_user['date_start'] = pd.to_datetime(dfy_user['date_start'], errors='coerce')
dfy_user.reset_index(drop=True, inplace=True)

dfy_user['year'] = dfy_user['date_start'].dt.year
dfy_user['month'] = dfy_user['date_start'].dt.month

dfy_user['source'] = 'yelp'
dfy_user.info()
dfy_user.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   reviews_count  int64         
 2   date_start     datetime64[ns]
 3   rating_avg     float64       
 4   year           int32         
 5   month          int32         
 6   source         object        
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1), object(2)
memory usage: 96.4+ MB


,user_id,reviews_count,date_start,rating_avg,year,month,source
1224345,5BlfVarhU1erq-PMPGStqg,3,2011-05-07 13:17:59,3.00,2011,5,yelp
32185,mFj4BgkDgq4wcowBl3UXwg,20,2009-06-14 16:12:52,4.19,2009,6,yelp
2055997,-siBzHBNqMCH_trQDnFbTg,82,2011-03-14 03:48:48,4.20,2011,3,yelp
639093,rSTt0kdSs1lBakZ_ShGrGA,4,2014-09-12 22:11:18,3.00,2014,9,yelp
43638,OumwiIT9PQtfvMJifqjz4w,64,2010-08-07 22:49:36,3.76,2010,8,yelp


#### YELP checkin

In [20]:
print(yelp_checkins.shape)
yelp_checkins.sample(2)

(28668325, 19)


,business_id,date,name,city,postal_code,latitude,longitude,stars,review_count,is_open,state,state_city,city_postalcode,state_city_postalcode,categories,food,price,year,month
25271298,sRJFxyCjZnmXuZLk6uKW6Q,2014-04-21 16:30:00,Panera Bread,Newark,19702,39.678257,-75.651854,2.5,35,0,DE,DE - Newark,Newark - 19702,DE - Newark - 19702,Cafes,yes,1.333333,2014,4
22457173,luuxLQDw4zsgbJiOPhARSg,2018-03-24 17:23:51,ALDI,Citrus Park,33626,28.043039,-82.581967,4.0,25,1,FL,FL - Citrus Park,Citrus Park - 33626,FL - Citrus Park - 33626,Specialty Food,yes,1.555556,2018,3


In [21]:
dfy_checkins = yelp_checkins[['business_id', 'date']]
dfy_checkins = dfy_checkins.rename(columns={'business_id':'site_id', 'date':'datetime'})

#dfy_checkins['datetime'] = pd.to_datetime(dfy_checkins['date'])
dfy_checkins['year'] = dfy_checkins['datetime'].dt.year
dfy_checkins['month'] = dfy_checkins['datetime'].dt.month

dfy_checkins['source'] = 'yelp'

dfy_checkins = dfy_checkins[['source', 'site_id', 'datetime', 'year', 'month']]

In [22]:
print(dfy_checkins.info())
dfy_checkins.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 28668325 entries, 0 to 28715821
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   source    object        
 1   site_id   object        
 2   datetime  datetime64[ns]
 3   year      int32         
 4   month     int32         
dtypes: datetime64[ns](1), int32(2), object(2)
memory usage: 1.1+ GB
None


,source,site_id,datetime,year,month
16203770,yelp,Yu_QofgDAjn__QsMi5Wudg,2019-01-19 19:11:20,2019,1
5525319,yelp,BJOGo_upuBElDT_xOaurIA,2014-02-13 02:31:10,2014,2


#### YELP categories

In [23]:
yelp_site_categories.info()
yelp_site_categories.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149643 entries, 0 to 149642
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   site_id     149643 non-null  object
 1   categories  149643 non-null  object
 2   food        149643 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


,site_id,categories,food
66626,qBnbXm2VhesKhmdRejnhSw,Bars,yes
105345,GHrXjivwy75GdaDxhneKyg,Chicken Wings,yes


In [24]:
dfy_categories = yelp_site_categories[['site_id', 'categories']]
dfy_categories['source'] = 'yelp'
dfy_categories = dfy_categories[['source', 'site_id', 'categories']]

dfy_categories.info()
dfy_categories.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149643 entries, 0 to 149642
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   source      149643 non-null  object
 1   site_id     149643 non-null  object
 2   categories  149643 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


,source,site_id,categories
53378,yelp,lfbRfx3RNwYZ-VaRG0vsqQ,Italian
116057,yelp,22JMnG7XzPRJC0aj-ipHtg,Convenience Stores


#### YELP attributes

In [25]:
dfy_attributes = yelp_site_attributes.copy()

print(dfy_attributes.shape)
dfy_attributes.sample(2)

(1206820, 2)


,business_id,attributes
129026,DYr-sOvFnfEfdHxxvwS3Yw,RestaurantsPriceRange2
90897,Jg9kkOl4sx2AXJVohbpSiw,RestaurantsGoodForGroups


### Datasets GOOGLE

#### GOOGLE reviews

In [26]:
print(google_reviews.info())
google_reviews.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346287 entries, 0 to 4346286
Data columns (total 7 columns):
 #   Column           Dtype         
---  ------           -----         
 0   user_id          float64       
 1   gmap_id          object        
 2   state            object        
 3   datetime         datetime64[ns]
 4   rating           int64         
 5   vader_polarity   float64       
 6   vader_sentiment  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 232.1+ MB
None


,user_id,gmap_id,state,datetime,rating,vader_polarity,vader_sentiment
2491073,1.131387e+20,0x89d36cf78a88eb49:0x24b1c7f1b2e64bdc,NY,2020-12-08 14:51:25.743,4,0.7845,1
1811574,1.140291e+20,0x8820fe24d8afedb5:0x92121a1815f036c8,MI,2020-03-07 14:55:11.010,5,0.0772,1


In [27]:
dfg_reviews = google_reviews
dfg_reviews['source'] = 'google'
dfg_reviews['month'] = dfg_reviews['datetime'].dt.month
dfg_reviews['year'] = dfg_reviews['datetime'].dt.year
dfg_reviews = dfg_reviews[['source', 'gmap_id','user_id', 'datetime', 'month', 'year', 'rating', 'vader_polarity', 'vader_sentiment']]
dfg_reviews = dfg_reviews.rename(columns={'gmap_id': 'site_id', 'vader_polarity':'polarity', 'vader_sentiment':'sentiment'})

In [28]:
dfg_reviews.sample(2)

,source,site_id,user_id,datetime,month,year,rating,polarity,sentiment
1732376,google,0x89e3a4309690cf97:0xa975c66a71d4939f,1.152199e+20,2018-09-17 06:01:39.763,9,2018,4,0.4404,1
3363171,google,0x89c5726d88bb1451:0x93cf3c0ff2b3b121,1.140947e+20,2017-05-28 11:40:50.970,5,2017,4,0.8257,1


#### GOOGLE restaurants

In [29]:
print(google_rest.info())
google_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 223485 entries, 2 to 274996
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            223485 non-null  object 
 1   address_full    223485 non-null  object 
 2   gmap_id         223485 non-null  object 
 3   latitude        223485 non-null  float64
 4   longitude       223485 non-null  float64
 5   avg_rating      223485 non-null  float64
 6   num_of_reviews  223485 non-null  int64  
 7   price           223485 non-null  float64
 8   address         223485 non-null  object 
 9   city            223485 non-null  object 
 10  postal_code     223485 non-null  object 
 11  state           223485 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 22.2+ MB
None


,name,address_full,gmap_id,latitude,longitude,avg_rating,num_of_reviews,price,address,city,postal_code,state
186042,M C's Sports Bar & Grill,"M C's Sports Bar & Grill, 11050 Co Rd V, Glen ...",0x87fcbe8eb51ee795:0xfe85d61a6c16115f,42.815865,-90.966022,4.6,28,1.0,11050 Co Rd V,Glen Haven,53810,WI
17359,Iguana,"Iguana, 5873 Albemarle Rd, Charlotte, NC 28212",0x885421ddb019b503:0xe19950c6180acf21,35.203945,-80.745768,3.9,78,1.0,5873 Albemarle Rd,Charlotte,28212,NC


In [30]:
dfg_rest = google_rest
dfg_rest['source'] = 'google'
dfg_rest = dfg_rest[['source','gmap_id', 'name', 'state', 'city', 'postal_code', 'price', 'avg_rating', 'num_of_reviews']]
dfg_rest = dfg_rest.rename(columns={'gmap_id': 'site_id', 'avg_rating':'rating_avg', 'num_of_reviews':'reviews_count'})

# Agregado de fecha de inicio de acitividad
dfg_rest_date_start = dfg_reviews.groupby('site_id')['datetime'].min().reset_index()
dfg_rest = pd.merge(dfg_rest, dfg_rest_date_start, how='left', on=['site_id'])
dfg_rest = dfg_rest.rename(columns={'datetime': 'date_start'})

dfg_rest['year'] = dfg_rest['date_start'].dt.year
dfg_rest['month'] = dfg_rest['date_start'].dt.month

dfg_rest['state_city'] = dfg_rest['state'].str.cat(dfg_rest['city'], sep=' - ')
dfg_rest['city_postalcode'] = dfg_rest['city'].str.cat(dfg_rest['postal_code'], sep=' - ')
dfg_rest['state_city_postalcode'] = dfg_rest['state'].str.cat(dfy_rest['city'], sep=' - ').str.cat(dfg_rest['postal_code'], sep=' - ')

In [31]:
print(dfg_rest.info())
dfg_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223485 entries, 0 to 223484
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   source                 223485 non-null  object        
 1   site_id                223485 non-null  object        
 2   name                   223485 non-null  object        
 3   state                  223485 non-null  object        
 4   city                   223485 non-null  object        
 5   postal_code            223485 non-null  object        
 6   price                  223485 non-null  float64       
 7   rating_avg             223485 non-null  float64       
 8   reviews_count          223485 non-null  int64         
 9   date_start             134644 non-null  datetime64[ns]
 10  year                   134644 non-null  float64       
 11  month                  134644 non-null  float64       
 12  state_city             223485 non-null  obje

,source,site_id,name,state,city,postal_code,price,rating_avg,reviews_count,date_start,year,month,state_city,city_postalcode,state_city_postalcode
89802,google,0x89c2bdf94c2cf1cf:0xda6bb9d5dcb6c063,Casa Tequila,NY,Armonk,10504,2.000000,4.5,55,2019-12-21 19:24:39.897,2019.0,12.0,NY - Armonk,Armonk - 10504,NY - Levittown - 10504
3404,google,0x8816154726998a07:0xc73acc478de0ab39,Knight & Day Sandwich Pub,IN,Kendallville,46755,1.333333,4.7,8,NaT,NaN,NaN,IN - Kendallville,Kendallville - 46755,IN - Saint Petersburg - 46755


#### GOOGLE user (creación de dataset)

In [32]:
dfg_user = dfg_reviews.groupby(['user_id']).agg({'site_id': 'count', 'datetime': 'min', 'rating': 'mean'}).reset_index()
dfg_user = dfg_user.rename(columns={'site_id':'reviews_count', 'datetime':'date_start', 'rating':'rating_avg'})
dfg_user['year'] = dfg_user['date_start'].dt.year
dfg_user['month'] = dfg_user['date_start'].dt.month

dfg_user['source'] = 'google'
dfg_user.sample(2)

,user_id,reviews_count,date_start,rating_avg,year,month,source
721414,1.051675e+20,1,2019-10-11 17:17:54.153,5.0,2019,10,google
2225911,1.155223e+20,3,2017-05-07 16:47:54.650,4.0,2017,5,google


#### GOOGLE checkins (creación de dataset)

In [33]:
dfy_checkins = yelp_checkins[['business_id', 'date']]
dfy_checkins = dfy_checkins.rename(columns={'business_id':'site_id', 'date':'datetime'})

#dfy_checkins['datetime'] = pd.to_datetime(dfy_checkins['date'])
dfy_checkins['year'] = dfy_checkins['datetime'].dt.year
dfy_checkins['month'] = dfy_checkins['datetime'].dt.month

dfy_checkins['source'] = 'yelp'

dfy_checkins = dfy_checkins[['source', 'site_id', 'datetime', 'year', 'month']]

In [34]:
dfg_checkins = dfg_reviews[['source', 'site_id', 'datetime', 'year', 'month']]

In [35]:
print(dfg_checkins.info())
dfg_checkins.sample(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346287 entries, 0 to 4346286
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   source    object        
 1   site_id   object        
 2   datetime  datetime64[ns]
 3   year      int32         
 4   month     int32         
dtypes: datetime64[ns](1), int32(2), object(2)
memory usage: 132.6+ MB
None


,source,site_id,datetime,year,month
1326030,google,0x87ee93e7e4732b31:0x1292f38eb8babd19,2019-03-17 11:18:31.839,2019,3
4329619,google,0x8765b30194472e1b:0x34f651ccb017a084,2020-07-26 10:28:16.142,2020,7


#### GOOGLE categories

In [36]:
google_site_categories.info()
google_site_categories.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 626995 entries, 2 to 274996
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   site_id   626995 non-null  object
 1   category  626995 non-null  object
dtypes: object(2)
memory usage: 14.4+ MB


,site_id,category
185966,0x89c4353c18175d6d:0xa396afe61f85317f,Chinese restaurant
163550,0x89dfd1af58f736fd:0x365f5734b5f8a9e4,Pizza restaurant


In [37]:
dfg_categories = google_site_categories.copy()

dfg_categories.rename(columns={'category': 'categories'}, inplace=True)
dfg_categories['source'] = 'google'
dfg_categories = dfg_categories[['source', 'site_id', 'categories']]

dfg_categories.info()
dfg_categories.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 626995 entries, 2 to 274996
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   source      626995 non-null  object
 1   site_id     626995 non-null  object
 2   categories  626995 non-null  object
dtypes: object(3)
memory usage: 19.1+ MB


,source,site_id,categories
167066,google,0x52b332d54812701b:0xf1a957b769a72641,Espresso bar
137178,google,0x883be0fa7c8c6dd1:0xa78ebbfbad44333a,Takeout Restaurant


### Datasets COMBINADOS

#### Dataset RESTAURANTS

In [38]:
# Filtrado
dfy_rest = dfy_rest[dfy_rest['site_id'].isin(dfgy_rest_uniques['business_id'])]
# Union
dfgy_rest = pd.concat([dfy_rest, dfg_rest])

print(dfgy_rest.info())
dfgy_rest.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 342834 entries, 3 to 223484
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   source                 342834 non-null  object        
 1   site_id                342834 non-null  object        
 2   name                   342834 non-null  object        
 3   state                  342834 non-null  object        
 4   city                   342834 non-null  object        
 5   postal_code            342834 non-null  object        
 6   price                  342834 non-null  float64       
 7   rating_avg             342834 non-null  float64       
 8   reviews_count          342834 non-null  int64         
 9   date_start             247726 non-null  datetime64[ns]
 10  year                   247726 non-null  float64       
 11  month                  247726 non-null  float64       
 12  state_city             342834 non-null  object   

,source,site_id,name,state,city,postal_code,price,rating_avg,reviews_count,date_start,year,month,state_city,city_postalcode,state_city_postalcode
137375,google,0x88c290243ba6978f:0x4a53ff164abdab4d,Golden Tequila Mexican Restaurant,FL,Port Richey,34652,1.454545,4.0,18,NaT,NaN,NaN,FL - Port Richey,Port Richey - 34652,FL - Beech Grove - 34652
1394,google,0x7c007278edb2a865:0x4ed1c3d61fda94aa,Tucker & Bevvy Breakfast,HI,Honolulu,96815,1.833333,4.2,57,2015-12-25 07:48:45.188,2015.0,12.0,HI - Honolulu,Honolulu - 96815,HI - New Orleans - 96815


#### Dataset USER

In [39]:
dfgy_user = pd.concat([dfy_user, dfg_user])

print(dfgy_user.info())
dfgy_user.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 4756911 entries, 0 to 2651313
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        object        
 1   reviews_count  int64         
 2   date_start     datetime64[ns]
 3   rating_avg     float64       
 4   year           int32         
 5   month          int32         
 6   source         object        
dtypes: datetime64[ns](1), float64(1), int32(2), int64(1), object(2)
memory usage: 254.0+ MB
None


,user_id,reviews_count,date_start,rating_avg,year,month,source
578667,104190924870702530560.0,5,2019-10-15 10:42:40.633,5.00,2019,10,google
209712,ui8mUkv49uhVchhFTjO3eg,13,2012-02-20 05:10:19.000,3.93,2012,2,yelp


#### Dataset CHECKINS

In [40]:
# Filtrado
dfy_checkins = dfy_checkins[dfy_checkins['site_id'].isin(dfgy_rest_uniques['business_id'])]

# Union
dfgy_checkins = pd.concat([dfy_checkins, dfg_checkins])

print(dfgy_checkins.info())
dfgy_checkins.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 28129090 entries, 0 to 4346286
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   source    object        
 1   site_id   object        
 2   datetime  datetime64[ns]
 3   year      int32         
 4   month     int32         
dtypes: datetime64[ns](1), int32(2), object(2)
memory usage: 1.0+ GB
None


,source,site_id,datetime,year,month
20895584,yelp,iOFbMitGdfU6i5xm4nZ8Ww,2016-12-27 18:34:22,2016,12
23911669,yelp,pCqAtxJDMv7ha57hOkB11w,2013-12-04 05:48:59,2013,12


#### Dataset REVIEWS

In [41]:
# Filtrado
dfy_reviews = dfy_reviews[dfy_reviews['site_id'].isin(dfgy_rest_uniques['business_id'])]

# Union
dfgy_reviews = pd.concat([dfy_reviews, dfg_reviews])

print(dfgy_reviews.info())
dfgy_reviews.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 5204599 entries, 0 to 4346286
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   source     object        
 1   site_id    object        
 2   user_id    object        
 3   datetime   datetime64[ns]
 4   month      int32         
 5   year       int32         
 6   rating     float64       
 7   polarity   float64       
 8   sentiment  int64         
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(3)
memory usage: 357.4+ MB
None


,source,site_id,user_id,datetime,month,year,rating,polarity,sentiment
3185808,google,0x54955e82a5b53587:0x4b2f7a547aa5e76c,108390412652622839808.0,2018-07-13 23:51:19.753,7,2018,1.0,-0.9121,-1
3201765,google,0x54951a450a6a0661:0x7a7c930df64ebddb,115430852941586087936.0,2020-07-07 18:20:31.023,7,2020,5.0,0.0000,0


#### Dataset CATEGORIES

In [42]:
# Filtrado
dfy_categories = dfy_categories[dfy_categories['site_id'].isin(dfgy_rest_uniques['business_id'])]

# Union
dfgy_categories = pd.concat([dfy_categories, dfg_categories])

print(dfgy_categories.info())
dfgy_categories.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 744630 entries, 0 to 274996
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   source      744630 non-null  object
 1   site_id     744630 non-null  object
 2   categories  744630 non-null  object
dtypes: object(3)
memory usage: 22.7+ MB
None


,source,site_id,categories
226663,google,0x87cbc08a2ceddb0f:0xedb583beb553fcb6,Restaurant
11241,google,0x54bff550f15e7a71:0x92b66f05da2a6240,Pizza restaurant


### Exportación de los datasets

In [45]:
# Restaurantes
dfgy_rest.to_parquet("../Data/Final/dfgy_rest.parquet")

# Usuarios
dfgy_user['user_id'] = dfgy_user['user_id'].astype(str)
dfgy_user.to_parquet("../Data/Final/dfgy_user.parquet")

# Checkins
dfgy_checkins.to_parquet("../Data/Final/dfgy_checkins.parquet")

# Reviews
dfgy_reviews['user_id'] = dfgy_reviews['user_id'].astype(str)
dfgy_reviews.to_parquet("../Data/Final/dfgy_reviews.parquet")

# Categorías
dfgy_categories.to_parquet("../Data/Final/dfgy_categories.parquet")
